In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

def extract_dominant_color(image_path, k=3):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    kmeans = KMeans(n_clusters=k)
    kmeans.fit(image)

    dominant_color = kmeans.cluster_centers_[0]
    return dominant_color

image_path = "path/to/your/image.jpg"
dominant_color = extract_dominant_color(image_path)
print("（RGB）:", dominant_color)